## Gdsfactory v8 Upgrade

Gdsfactory v8, based on KFactory, offers enhanced routing functions and additional features from KLayout, including DRC, dummy fill, and connectivity checks.

For those still using gdsfactory v7, it is hosted in [https://github.com/gdsfactory/gdsfactory7](https://github.com/gdsfactory/gdsfactory7), along with the [documentation](https://gdsfactory.github.io/gdsfactory7/).

### Benefits of Migrating:

- **Integrated Data**: Ports, information, and settings are stored within the GDS file, eliminating the need for separate files.
- **Improved Booleans**: Booleans are more robust with integer-based polygons, removing slivers of less than 1nm.
- **Enhanced Features**: More robust booleans, DRC, LVS, and connectivity checks.
- **Active Maintenance**: KLayout is more actively maintained with frequent updates than gdstk.
- **Advanced Routing Algorithms**: Better routing algorithms for efficient design. Thanks to Kfactory.
- **Grid Alignment**: Ports and polygons snap to grid by default, reducing the likelihood of 1nm gaps.

### Drawbacks of Migrating:

- **Potential Errors**: As with any code changes, undesired errors may be introduced. It is recommended to have regression tests for all your components before you migrate.
- **Non-Manhattan Placement**: Slightly more difficult to define non-manhattan references, routes, or port. Manhattan Components (at 0, 90, 180 and 270) work the same way.
- **Incomplete Functionality**: Some features, such as `route_path_length_match`, are not yet implemented.

### Major Differences:

- **Coordinate System**: `xmin`, `xmax`, `ymin`, and `ymax` are deprecated and will change to Database Units (1nm by default) in gdsfactory9. To set/get them in um, use `d` (decimal) e.g., `Instance.dxmin` which is in float.
- **LayerMap**: Now an Enum of integers.
- **Routing Functions**: New functions do not require starting ports to be in the same orientation and monitor for self-intersections. `get_route` is now `route_single`, and `get_bundle` is now `route_bundle`.
- **Grid Snapping**: All polygon points snap to grid, mitigating 1nm gaps.

### Minor Differences:

- Replace `from gdsfactory.cell import cell` with `from gdsfactory import cell`.

### How to Migrate:

We provide a migration script to assist with migrating your code. Ensure to verify any automatic migrations, and ideally, write regression tests for your code before migrating to ensure accuracy.

```bash
gf migrate -i --migration 7to8 <input_folder/file> 
# or 
gf migrate --inplace --migration 7to8 <input_folder/file>


## LayerMap 

In v7 or below, a LayerMap needs to be called

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab()
```

However in v8 it has a different type and does not need to be called 

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab
```

See below:

In [ ]:
import gdsfactory as gf
from gdsfactory.technology import LayerMap


class LayerMapFab(LayerMap):
    WG = (1, 0)


LAYER = LayerMapFab
type(LAYER)

In [ ]:
LAYER.WG

In [ ]:
tuple(LAYER.WG)

In [ ]:
str(LAYER.WG)

## Component.bbox is now a function

Both Components and Instances have a `bbox` and `dbbox` that are now functions.

In [ ]:
c = gf.c.straight(length=10)
c.bbox()

In [ ]:
dir(c.dbbox())

In [ ]:
c.dbbox().right

For the old bbox behavior you can use `c.bbox_np` which returns the bbox as a numpy array.

In [ ]:
c.bbox_np()

## Component.get_polygons()

`Component.get_polygons()` returns all the Polygons and can be slow.

`Component.get_polygons_points()` is the equivalent to `Component.get_polygons()` in gdsfactory7 where we return the polygon edges.


In [ ]:
import gdsfactory as gf

c = gf.pack([gf.components.seal_ring_segmented()] * 200)[0]
c

In [ ]:
%time
p = c.get_polygons_points()

If you only care about the polygons from one layer you can also only extract those.

In [ ]:
%time
p = c.get_polygons_points(layers=("M3",))

## ref.get_ports_list

Reference does not have `get_ports_list`. You can use

- ref.ports to get all ports
- ref.ports.filter(...) to filter all ports with angle, port_type, orientation ...
- gf.port.get_ports_list(b, ...) for backwards compatibility.

In [ ]:
import gdsfactory as gf

c = gf.Component()
bend = gf.components.bend_euler()
b = c.add_ref(bend)

b.ports.filter(orientation=90)

In [ ]:
gf.port.get_ports_list(b, orientation=90)

## Routing functions

Routing functions NO longer return the route Instances but they place the instances in a Component, so you have to pass a Component as the first argument.

In [ ]:
c = gf.Component()
w = gf.components.straight(cross_section="rib")
top = c << w
bot = c << w
bot.dmove((0, -2))

p0 = top.ports["o2"]
p1 = bot.ports["o2"]

r = gf.routing.route_single(
    c,
    p0,
    p1,
    cross_section="rib",
)
c

🚀 The new routing functions allow the starting ports `ports1` to have different orientations.

The end ports `ports2` still require to have the same orientation.

In [ ]:
c = gf.Component()
top = c << gf.components.nxn(north=8, south=0, east=0, west=0)
bot = c << gf.components.nxn(north=2, south=2, east=2, west=2, xsize=10, ysize=10)
top.dmovey(100)

routes = gf.routing.route_bundle(
    c,
    ports1=bot.ports,
    ports2=top.ports,
    radius=5,
    sort_ports=True,
    cross_section="strip",
)
c

## Built in connectivity checks

gdsfactory8 includes connectivity checks to ensure things are properly connected. no

This can help you find any:

- Gaps between ports.
- Width mismatches.
- Unconnected ports.


In [ ]:
c = gf.Component()
s = c << gf.components.straight(width=1)

b1 = c << gf.components.bend_euler()
b1.connect("o1", s["o2"], allow_width_mismatch=True)

b2 = c << gf.components.bend_euler(radius=5)
b2.connect("o1", s["o1"], allow_width_mismatch=True)

gc = gf.components.grating_coupler_elliptical_te()
gc1 = c << gc
gc2 = c << gc

gc1.connect("o1", b1.ports["o2"])
gc2.connect("o1", b2.ports["o2"])

c.plot()

In [ ]:
lyrdb = c.connectivity_check(port_types=("optical", "electrical"))
filepath = gf.config.home / "errors.lyrdb"
lyrdb.save(filepath)
gf.show(c, lyrdb=filepath)